In [1]:
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import sys
import time

import sknn
import theano
import theano.tensor as T
import lasagne
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import visualize

from functools import reduce

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn import svm
from six.moves import cPickle as pickle

from sklearn.neural_network import MLPClassifier
from collections import Counter

%matplotlib inline

C:\Users\Shadi\Anaconda3\lib\site-packages\theano\tensor\signal\downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
C:\Users\Shadi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Import libraries

In [2]:
files = [
    'data_batch_1',
    'data_batch_2',
    'data_batch_3',
    'data_batch_4',
    'data_batch_5',
    'test_batch'
]
data = []
labels = []
start = time.time()
for file in files:
    with open(file, 'rb') as f:
        d = pickle.load(f, encoding='bytes')
        if file == 'test_batch':
            test_data = d[b'data']
            test_labels = d[b'labels']
        else:
            data.append(d[b'data'])
            labels.append(d[b'labels'])
end = time.time()
print('Time to load data: {:.3f}s'.format(end - start))
for i in range(len(data)):
    print('Train data {}:'.format(i), data[i].shape, len(labels[i]))
print('Test data:', test_data.shape, len(test_labels))

merged_data = reduce(lambda a,b: np.vstack((a,b)), data)
merged_labels = reduce(lambda a,b: a+b, labels)
print('Merged train data:', merged_data.shape, len(merged_labels))

Time to load data: 0.518s
Train data 0: (10000, 3072) 10000
Train data 1: (10000, 3072) 10000
Train data 2: (10000, 3072) 10000
Train data 3: (10000, 3072) 10000
Train data 4: (10000, 3072) 10000
Test data: (10000, 3072) 10000
Merged train data: (50000, 3072) 50000


Load data and print out time- this code is taken directly from section

In [3]:
scaler = StandardScaler()
scaler.fit(merged_data) 
#Train our scaler based on our training data

train_data = merged_data[:]
train_labels = merged_labels[:]
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
#Apply our scaling to our training and test data, creating a copy of our merged data called 'train_data'

C:\Users\Shadi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Scaling all the data so the Neural Network will work as expected

In [4]:
mlp = MLPClassifier(solver='lbgfs', alpha=1e-5, hidden_layer_sizes=(5,2), random_state=1)
start = time.time()
mlp.fit(train_data, train_labels)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))
#Create and train a basic Multi Layer Perceptron model

Time to build: 168.079s


In [5]:
predictions = mlp.predict(train_data) #Store the predictions for this basic model
correct = merged_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
accuracy = (correct == 0).sum() / len(correct)
accuracy * 100
#This is the accuracy of the base model on the training data

32.013999999999996

In [6]:
test_pred = mlp.predict(test_data)
correct = test_labels - test_pred #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Accuracy of the base model on the test data

30.34

Our basic MLP model seems to have an accuracy of roughly 30-32% to start- looks like there's a lot of optimization we can do on this

http://lasagne.readthedocs.io/en/latest/user/tutorial.html

The documentation for the Lasagne package was extensively utilized for creating the following Neural Network.

In [7]:
start = time.time()
svmc = svm.SVC()
svmc.fit(train_data, train_labels)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))

Time to build: 7249.084s


In [9]:
predictions = svmc.predict(test_data)
correct = test_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100

54.810000000000002

SVM at 54.8% accuracy- much higher than the 30.34% of our MLP Neural Network

Now we're going to try varying the size of the training set.

In [10]:
Counter(train_labels)
#We have 5,000 of each lable in our data

bucket = []
indices = []
for i in range(0,10):
    temp = [z for z, x in enumerate(train_labels) if x == i]
    indices.append(temp)
    bucket.append(indices[i])
#This finds all the indexes for each value in our training data, and stores them into the 'bucket' list of lists

In [11]:
t1 = []
l1 = []

t2 = []
l2 = []

t3 = []
l3 = []
#Create our training and label subsets

c12 = []
c13 = []
c23 = []
#Create our combined training sets

v12 = []
v13 = []
v23 = []
#Create our combined label sets

for i in range(0,10):
    t1.extend(bucket[i][:1700])
    l1.extend(bucket[i][:1700])
    
    t2.extend(bucket[i][1701:3400])
    l2.extend(bucket[i][1701:3400])
    
    t3.extend(bucket[i][3401:5000])
    l3.extend(bucket[i][3401:5000])
    #Now we append those index subsets together. This is for our basic sets.
    
    c12.extend(bucket[i][:3400])
    c13.extend(bucket[i][:1700])
    c13.extend(bucket[i][3401:5000])
    c23.extend(bucket[i][1701:5000])
    
    v12.extend(bucket[i][:3400])
    v13.extend(bucket[i][:1700])
    v13.extend(bucket[i][3401:5000])
    v23.extend(bucket[i][1701:5000])
    #Now for the combined sets

t1 = train_data[np.array(t1)]
l1 = [train_labels[i] for i in l1]

t2 = train_data[np.array(t2)]
l2 = [train_labels[i] for i in l2]

t3 = train_data[np.array(t3)]
l3 = [train_labels[i] for i in l3]
#And now we convert them so they're actually lists of values rather than indices
#Above is simple sets

c12 = train_data[np.array(c12)]
v12 = [train_labels[i] for i in v12]

c13 = train_data[np.array(c13)]
v13 = [train_labels[i] for i in v13]

c23 = train_data[np.array(c23)]
v23 = [train_labels[i] for i in v23]

Now we've split our data into the following sets:

T1/T2/T3: Training data, in thirds, of the original set- each containing the same number of each class
C12/C13/C23: Combined versions of the above three sets. We've already used ALL the data in a run, so there's no point to C123

In [12]:
start = time.time()
mlp.fit(t1, l1)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))
#Basic MLP model with the first test set

Time to build: 61.733s


In [13]:
test_pred = mlp.predict(test_data)
correct = test_labels - test_pred #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Accuracy of the base model on the test data

27.350000000000001

In [14]:
start = time.time()
mlp.fit(t2, l2)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))
#Basic MLP model with the second test set

Time to build: 61.143s


In [15]:
test_pred = mlp.predict(test_data)
correct = test_labels - test_pred #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Accuracy of the base model on the test data

27.039999999999999

In [16]:
start = time.time()
mlp.fit(t3, l3)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))
#Basic MLP model with the third test set

Time to build: 57.365s


In [17]:
test_pred = mlp.predict(test_data)
correct = test_labels - test_pred #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Accuracy of the base model on the test data

27.089999999999996

In [18]:
start = time.time()
mlp.fit(c12, v12)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))
#Basic MLP model with the first combined set

Time to build: 116.076s


In [19]:
test_pred = mlp.predict(test_data)
correct = test_labels - test_pred #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Accuracy of the base model on the test data

29.210000000000001

In [20]:
start = time.time()
mlp.fit(c13, v13)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))
#Basic MLP model with the second combined set

Time to build: 116.821s


In [21]:
test_pred = mlp.predict(test_data)
correct = test_labels - test_pred #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Accuracy of the base model on the test data

28.360000000000003

In [22]:
start = time.time()
mlp.fit(c23, v23)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))
#Basic MLP model with the final combined set

Time to build: 113.407s


In [23]:
test_pred = mlp.predict(test_data)
correct = test_labels - test_pred #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Accuracy of the base model on the test data

29.160000000000004

This finishes our comparison of the MLP classifier- next we have to compare our SVM classifier. Generally speaking the classifiers took about 1-2 minutes to build- looks like my computer runs at almost 1 minute/17,000 records for the training data.

In [24]:
start = time.time()
svmc.fit(t1, l1)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))

Time to build: 908.899s


In [25]:
predictions = svmc.predict(test_data)
correct = test_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#First set

49.919999999999995

In [26]:
start = time.time()
svmc.fit(t2, l2)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))

Time to build: 893.248s


In [27]:
predictions = svmc.predict(test_data)
correct = test_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Second set

49.710000000000001

In [28]:
start = time.time()
svmc.fit(t3, l3)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))

Time to build: 807.029s


In [29]:
predictions = svmc.predict(test_data)
correct = test_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Third set

49.669999999999995

In [ ]:
start = time.time()
svmc.fit(c12, v12)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))

In [ ]:
predictions = svmc.predict(test_data)
correct = test_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#First Combo set

In [ ]:
start = time.time()
svmc.fit(c13, v13)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))

In [ ]:
predictions = svmc.predict(test_data)
correct = test_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Second Combo set

In [ ]:
start = time.time()
svmc.fit(c23, v23)
end = time.time()
print('Time to build: {:.3f}s'.format(end - start))

In [ ]:
predictions = svmc.predict(test_data)
correct = test_labels - predictions #If the numbers are the same, they'll be 0, otherwise- any other number
test_accuracy = (correct == 0).sum() / len(correct)
test_accuracy * 100
#Last Combo set